In [12]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math
# Graphics
import seaborn as sns ; sns.set()


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
# from sklearn.metrics import root_mean_squared_error

In [13]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT *
from "filmview"
where "runtimeMinutes" Is NOT null and "titleType" = 'movie'
limit 10000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt2288067,Thailand Torn Between Two Worlds,movie,0,2009.0,None,68,Documentary,8.9,9,"[self_Adam_King, self_Paul_Everett, editor_Bri..."
1,tt2288081,The Devil's Nightmare,movie,0,2012.0,None,100,Horror,6.3,33,"[actress_Kt_Baldassaro, producer_Chris_Shanaha..."
2,tt2288093,The Little Maestro,movie,0,2012.0,None,108,Drama,4.6,46,"[producer_Osamu_Fujita, producer_Takahiro_Suna..."
3,tt2288105,The Recording,movie,0,2014.0,None,82,Horror,6.7,10,"[actress_Nicole_L._Osborne, producer_Chris_Bos..."
4,tt2288107,The Screenwriters,movie,0,2016.0,None,76,"Comedy,Drama,Family",6.2,21,"[actress_Jenn_Gotzon, composer_Benjamin_Botkin..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,tt2719024,Beautiful Destroyer,movie,0,2015.0,None,115,"Action,Drama,Romance",6.3,41,"[actor_Andra_Fuller, director_Christopher_Dorr..."
9996,tt2719032,"And, There You Are",movie,0,2010.0,None,105,"Comedy,Drama,Romance",3.8,27,"[actor_Roy_Kirkland, actor_Jonathan_M._Dangler..."
9997,tt27190438,The Three of Us,movie,0,2023.0,None,78,Documentary,8.2,8,"[director_Henya_Brodbeker, producer_Avigail_Sp..."
9998,tt27190932,My Beautiful Man: Eternal,movie,0,2023.0,None,103,"Drama,Romance",7.5,290,"[actor_Yûsei_Yagi, director_Mai_Sakai, actor_A..."


In [14]:
def listTostr (df):
    return df.apply(lambda x: ' '.join(map(str, x)))

In [15]:
# df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
# df["startYear"].fillna(df["startYear"].mean(), inplace=True)
# df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
# df["isAdult"] = df["isAdult"].apply(lambda x: 1 if x == "True" else 0)
df['isAdult'] = df['isAdult'].astype(bool)
df['genres'].fillna('missing', inplace=True)
df['Cate&names'].fillna('missing', inplace=True)
df['Cate&names']=df[['Cate&names']].apply(listTostr)
df


,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt2288067,Thailand Torn Between Two Worlds,movie,False,2009.0,None,68,Documentary,8.9,9,self_Adam_King self_Paul_Everett editor_Bridge...
1,tt2288081,The Devil's Nightmare,movie,False,2012.0,None,100,Horror,6.3,33,actress_Kt_Baldassaro producer_Chris_Shanahan ...
2,tt2288093,The Little Maestro,movie,False,2012.0,None,108,Drama,4.6,46,producer_Osamu_Fujita producer_Takahiro_Sunazu...
3,tt2288105,The Recording,movie,False,2014.0,None,82,Horror,6.7,10,actress_Nicole_L._Osborne producer_Chris_Boss ...
4,tt2288107,The Screenwriters,movie,False,2016.0,None,76,"Comedy,Drama,Family",6.2,21,actress_Jenn_Gotzon composer_Benjamin_Botkin a...
...,...,...,...,...,...,...,...,...,...,...,...
9995,tt2719024,Beautiful Destroyer,movie,False,2015.0,None,115,"Action,Drama,Romance",6.3,41,actor_Andra_Fuller director_Christopher_Dorrah...
9996,tt2719032,"And, There You Are",movie,False,2010.0,None,105,"Comedy,Drama,Romance",3.8,27,actor_Roy_Kirkland actor_Jonathan_M._Dangler a...
9997,tt27190438,The Three of Us,movie,False,2023.0,None,78,Documentary,8.2,8,director_Henya_Brodbeker producer_Avigail_Sper...
9998,tt27190932,My Beautiful Man: Eternal,movie,False,2023.0,None,103,"Drama,Romance",7.5,290,actor_Yûsei_Yagi director_Mai_Sakai actor_Akir...


In [16]:
# Séparation des caractéristiques et de la cible
X = df.drop(columns=["averageRating"])
y = df["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [17]:
# Définition des colonnes numériques, textuelles et de description
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = ['titleType']#['titleType', 'directors', 'writers']
genre = 'genres'
description_feature = 'Cate&names'

In [18]:
# Création des transformers pour les colonnes numériques, booléennes, textuelles et de description
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1)) 

text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('vect', CountVectorizer())
])

# Création d'un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', text_transformer, text_features),
        ('genre', Vect_transformer, genre),
        ('description', Vect_transformer, description_feature)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['startYear', 'runtimeMinutes']),
                                ('bool',
                                 FunctionTransformer(func=<function <lambda> at 0x000001B5ABA7B420>),
                                 'isAdult'),
                                ('text',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['titleType']),
                                ('genre',
                                 Pipeline(steps=[('vect', CountVectorizer())]),
                                 'genres'),
                                ('description',
                                 Pipeline(steps=[('vect', CountVectorizer())]),
                                 'Cate&names')])

In [19]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 10.0], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [50, 100], 'regressor__max_depth': [10, 20]},# None, 
    'Gradient Boosting Regression': {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [3, 5, 7]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


In [20]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):
  # Scoring
  #multi_scoring = {mean_squared_error,r2_score}
    
    
  # Grid search
  grid = GridSearchCV(pipeline, parameters,  scoring=scoring, refit='RMSE', cv=cv, n_jobs =-1, verbose = 1, error_score='raise')

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

In [21]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")
 
    
    

In [22]:

# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)
    
    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")
 


..............Linear Regression..............................
Fitting 5 folds for each of 2 candidates, totalling 10 fits
LinearRegression training time: 4.1168
Best LinearRegression parameters: {'regressor__fit_intercept': True}
Best LinearRegression score: 2.386
Linear Regression RMSE: 1.4365

..............Ridge Regression..............................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Ridge training time: 1.6303
Best Ridge parameters: {'regressor__alpha': 1.0}
Best Ridge score: 1.3125
Ridge Regression RMSE: 1.3189

..............Lasso Regression..............................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Lasso training time: 3.0896
Best Lasso parameters: {'regressor__alpha': 0.1}
Best Lasso score: 1.4174
Lasso Regression RMSE: 1.4242

..............ElasticNet..............................
Fitting 5 folds for each of 9 candidates, totalling 45 fits
ElasticNet training time: 3.6186
Best ElasticNet parameters: {'regressor__alpha

In [ ]:
'''
# Boucle sur les modèles pour ajuster avec GridSearchCV
results = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit='RMSE', cv=5, n_jobs =-1, verbose = 1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    results[model_name] = mse
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")
    print(f"{model_name} MSE: {mse}")
    print()
'''

'\n# Boucle sur les modèles pour ajuster avec GridSearchCV\nresults = {}\n\n# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation\nfor model_name, pipeline in pipelines.items():\n    print(f"Training {model_name}...")\n    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit=\'RMSE\', cv=5, n_jobs =-1, verbose = 1)\n    grid_search.fit(X_train, y_train)\n    best_model = grid_search.best_estimator_\n    y_pred = best_model.predict(X_test)\n    mse = root_mean_squared_error(y_test, y_pred)\n    results[model_name] = mse\n    print(f"Best parameters: {grid_search.best_params_}")\n    print(f"Best {model_name} score: {-grid_search.best_score_}")\n    print(f"{model_name} MSE: {mse}")\n    print()\n'

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")
    
'''

'\n# Evaluation des modèles sur les données de test\nprint("\nComparaison des performances des modèles sur les données de test:")\nfor model_name, mse in results.items():\n    print(f"{model_name}: MSE = {mse}")\n    \n'